In [1]:
# notebooks/08_inception_time.ipynb

import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import wfdb
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tqdm import tqdm

# 1. Veri Yolu Ayarları
DATA_PATH = Path("d:/ecg/data/raw/ptbxl")
SAMPLING_RATE = 500  # PTB-XL'in örnekleme hızı

In [2]:
# 2. InceptionTime Model Mimarisi
class InceptionBlock(nn.Module):
    def __init__(self, in_channels, n_filters=32, kernel_sizes=[9, 19, 39], bottleneck_channels=32):
        super().__init__()
        
        self.bottleneck = nn.Conv1d(in_channels, bottleneck_channels, 1) if bottleneck_channels else None
        
        # Paralel konvolüsyon katmanları
        self.convs = nn.ModuleList([
            nn.Conv1d(
                bottleneck_channels if bottleneck_channels else in_channels,
                n_filters,
                kernel_size,
                padding=kernel_size // 2
            ) for kernel_size in kernel_sizes
        ])
        
        # MaxPool yolu
        self.maxpool = nn.Sequential(
            nn.MaxPool1d(3, stride=1, padding=1),
            nn.Conv1d(in_channels, n_filters, 1)
        )
        
        self.bn = nn.BatchNorm1d(n_filters * (len(kernel_sizes) + 1))
        self.relu = nn.ReLU()
        
    def forward(self, x):
        if self.bottleneck:
            x = self.bottleneck(x)
        
        # Paralel yolları birleştir
        x_parallel = []
        for conv in self.convs:
            x_parallel.append(conv(x))
        x_parallel.append(self.maxpool(x))
        x = torch.cat(x_parallel, dim=1)
        
        return self.relu(self.bn(x))

class InceptionTime(nn.Module):
    def __init__(self, in_channels=12, n_classes=5, n_filters=32, n_blocks=6, kernel_sizes=[9, 19, 39]):
        super().__init__()
        
        self.blocks = nn.Sequential(*[
            InceptionBlock(
                in_channels if i == 0 else n_filters * 4,
                n_filters=n_filters,
                kernel_sizes=kernel_sizes
            ) for i in range(n_blocks)
        ])
        
        self.gap = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(n_filters * 4, n_classes)
        
    def forward(self, x):
        x = self.blocks(x)
        x = self.gap(x).squeeze(-1)
        return self.fc(x)

In [3]:
# 3. Veri Yükleme ve Ön İşleme
def load_ptbxl_data(data_path, sampling_rate=500):
    """PTB-XL veri setini yükle"""
    
    print(f"Veri yolu kontrolü: {data_path}")
    
    # Meta veriyi oku
    meta_path = data_path / "ptbxl_database.csv"
    if not meta_path.exists():
        raise FileNotFoundError(f"Meta veri dosyası bulunamadı: {meta_path}")
        
    df = pd.read_csv(meta_path)
    df.scp_codes = df.scp_codes.apply(eval)
    
    # Tanı sınıfları
    diagnostic_classes = ['NORM', 'MI', 'STTC', 'CD', 'HYP']
    
    # Etiketleri hazırla
    labels = np.zeros((len(df), len(diagnostic_classes)))
    for idx, row in df.iterrows():
        for diagnosis in row.scp_codes:
            if diagnosis in diagnostic_classes:
                labels[idx, diagnostic_classes.index(diagnosis)] = 1
    
    # Veri yükleme fonksiyonu
    def load_waveform(path):
        if not path.parent.exists():
            raise FileNotFoundError(f"Dizin bulunamadı: {path.parent}")
        if not (path.parent / f"{path.name}.hea").exists():
            raise FileNotFoundError(f"Header dosyası bulunamadı: {path}.hea")
            
        record = wfdb.rdrecord(str(path))
        return record.p_signal.T  # (leads, time_steps) formatına çevir
    
    # Tüm kayıtları yükle
    waveforms = []
    errors = []
    
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Kayıtlar yükleniyor"):
        try:
            # EKG kayıt yolu düzeltildi
            record_path = data_path / row['filename_hr'].replace('.hea', '')
            waveform = load_waveform(record_path)
            
            # Normalize et
            scaler = StandardScaler()
            waveform = scaler.fit_transform(waveform.T).T
            waveforms.append(waveform)
            
        except Exception as e:
            errors.append((idx, str(e)))
            if len(errors) < 5:  # İlk 5 hatayı göster
                print(f"\nHata (kayıt {idx}): {str(e)}")
                print(f"Dosya yolu: {record_path}")
    
    if errors:
        print(f"\nToplam {len(errors)} kayıt yüklenemedi")
    
    if not waveforms:
        raise ValueError("Hiç kayıt yüklenemedi!")
        
    print(f"\nBaşarıyla yüklenen kayıt sayısı: {len(waveforms)}")
    
    return np.array(waveforms), labels, diagnostic_classes

# Test et
try:
    print("\nVeri yükleniyor...")
    X, y, class_names = load_ptbxl_data(DATA_PATH)
    print("\nVeri yükleme başarılı!")
    print(f"X şekli: {X.shape}")
    print(f"y şekli: {y.shape}")
    print(f"Sınıflar: {class_names}")
    
except Exception as e:
    print(f"\nHata: {str(e)}")
    
    # Dizin yapısını kontrol et
    print("\nDizin yapısı:")
    if DATA_PATH.exists():
        print(f"\n{DATA_PATH} içeriği:")
        for item in DATA_PATH.glob("*"):
            print(f"- {item.name}")
            if item.is_dir():
                for subitem in item.glob("*"):
                    print(f"  - {subitem.name}")


Veri yükleniyor...
Veri yolu kontrolü: d:\ecg\data\raw\ptbxl


Kayıtlar yükleniyor:   1%|          | 147/21837 [00:04<07:08, 50.67it/s]


Hata (kayıt 136): Header dosyası bulunamadı: d:\ecg\data\raw\ptbxl\records500\00000\00137_hr.hea
Dosya yolu: d:\ecg\data\raw\ptbxl\records500\00000\00137_hr

Hata (kayıt 138): Header dosyası bulunamadı: d:\ecg\data\raw\ptbxl\records500\00000\00139_hr.hea
Dosya yolu: d:\ecg\data\raw\ptbxl\records500\00000\00139_hr

Hata (kayıt 139): Header dosyası bulunamadı: d:\ecg\data\raw\ptbxl\records500\00000\00140_hr.hea
Dosya yolu: d:\ecg\data\raw\ptbxl\records500\00000\00140_hr

Hata (kayıt 140): Header dosyası bulunamadı: d:\ecg\data\raw\ptbxl\records500\00000\00141_hr.hea
Dosya yolu: d:\ecg\data\raw\ptbxl\records500\00000\00141_hr


Kayıtlar yükleniyor: 100%|██████████| 21837/21837 [13:12<00:00, 27.57it/s]



Toplam 38 kayıt yüklenemedi

Başarıyla yüklenen kayıt sayısı: 21799

Veri yükleme başarılı!
X şekli: (21799, 12, 5000)
y şekli: (21837, 5)
Sınıflar: ['NORM', 'MI', 'STTC', 'CD', 'HYP']


In [ ]:
# 5. Eğitim Fonksiyonu
def train_model(model, train_loader, test_loader, n_epochs=50):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    
    criterion = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer,
        mode='min',
        factor=0.5,
        patience=5,
        verbose=True
    )
    
    best_loss = float('inf')
    history = {'train_loss': [], 'test_loss': [], 'train_acc': [], 'test_acc': []}
    
    for epoch in range(n_epochs):
        # Training
        model.train()
        train_loss = 0
        train_acc = 0
        
        for batch_idx, (data, target) in enumerate(tqdm(train_loader, desc=f'Epoch {epoch+1}')):
            data, target = data.to(device), target.to(device)
            
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            
            # Accuracy
            pred = torch.sigmoid(output) > 0.5
            train_acc += (pred == target).float().mean().item()
            
        train_loss /= len(train_loader)
        train_acc /= len(train_loader)
        
        # Testing
        model.eval()
        test_loss = 0
        test_acc = 0
        
        with torch.no_grad():
            for data, target in test_loader:
                data, target = data.to(device), target.to(device)
                output = model(data)
                test_loss += criterion(output, target).item()
                
                pred = torch.sigmoid(output) > 0.5
                test_acc += (pred == target).float().mean().item()
        
        test_loss /= len(test_loader)
        test_acc /= len(test_loader)
        
        # Learning rate scheduling
        scheduler.step(test_loss)
        
        # History
        history['train_loss'].append(train_loss)
        history['test_loss'].append(test_loss)
        history['train_acc'].append(train_acc)
        history['test_acc'].append(test_acc)
        
        print(f'\nEpoch {epoch+1}/{n_epochs}:')
        print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}')
        print(f'Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}')
        
        # Save best model
        if test_loss < best_loss:
            best_loss = test_loss
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': best_loss,
            }, 'best_inception_model.pt')
    
    return history